In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from library import datasets, models
from flax import linen as nn
from jax import random, numpy as jnp
import optax
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
MODEL_LATENT_DIM = 4
TRUE_LATENT_DIM = 2
AMBIENT_DIM = 2

In [15]:
key = random.PRNGKey(5)
key_A, key_B = random.split(key)
A = datasets.utils.make_blobs(n_samples=1000, min_sigma=0, max_sigma=0.1, key=key_A)
B = datasets.utils.make_blobs(n_samples=1000, min_sigma=0, max_sigma=0.1, key=key_B)

In [16]:
discriminator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(16),
    nn.relu,
    nn.Dense(16),
    nn.relu,
    nn.Dense(8),
    nn.relu,
    nn.Dense(1),
])

generator = nn.Sequential([
    nn.Dense(8),
    nn.relu,
    nn.Dense(8),
    nn.relu,
    nn.Dense(4),
    nn.relu,
    nn.Dense(AMBIENT_DIM),
])

model = models.cyclegan.CycleGAN(generator, discriminator, (AMBIENT_DIM,), (AMBIENT_DIM,))
#print(model)
model.initialize(optax.sigmoid_binary_cross_entropy)

In [17]:
gan_AB, gan_BA = model.create_distribution()

In [18]:
real_A = A.get_tensors()
real_B = B.get_tensors()
fake_A = gan_BA.draw_samples(real_B)
fake_B = gan_AB.draw_samples(real_A)
print(fake_A.shape, fake_B.shape, real_A.shape, real_B.shape)

(1000, 2) (1000, 2) (1000, 2) (1000, 2)


In [19]:
df_A = {'x': jnp.concatenate((real_A[:, 0], fake_A[:, 0]), axis=0),
      'y': jnp.concatenate((real_A[:, 1], fake_A[:, 1]), axis=0),
      'labels': jnp.concatenate((jnp.ones((real_A.shape[0],)), jnp.zeros((fake_A.shape[0],))), axis=0)}
px.scatter(df_A, x='x', y='y', color='labels')

In [21]:
g_loss, d_loss = model.train(real_A, real_B, optax.adam(learning_rate=1e-3), print_every=5, batch_size=1000, num_epochs=700)

0it [00:02, ?it/s]/700 [00:00<?, ?it/s]
  0%|          | 0/700 [00:05<?, ?it/s]


TypeError: unsupported operand type(s) for *: 'float' and 'FrozenDict'

In [19]:
gan_AB, gan_BA = model.create_distribution()

In [20]:
real_A = A.get_tensors()
real_B = B.get_tensors()
fake_A = gan_BA.draw_samples(real_B)
fake_B = gan_AB.draw_samples(real_A)
print(fake_A.shape, fake_B.shape, real_A.shape, real_B.shape)

(1000, 2) (1000, 2) (1000, 2) (1000, 2)


In [21]:
df_A = {'x': jnp.concatenate((real_A[:, 0], fake_A[:, 0]), axis=0),
      'y': jnp.concatenate((real_A[:, 1], fake_A[:, 1]), axis=0),
      'labels': jnp.concatenate((jnp.ones((real_A.shape[0],)), jnp.zeros((fake_A.shape[0],))), axis=0)}
px.scatter(df_A, x='x', y='y', color='labels')